In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Prepare for datapath

In [ ]:
import pandas as pd
import shutil
import os

!git clone https://github.com/ieee8023/covid-chestxray-dataset.git

!unzip -q '/content/drive/MyDrive/MEDICAL/Labeled Optical Coherence Tomography (OCT) and Chest X-Ray Images for Classification.zip' -d '/content/d/'
!unzip -q '/content/d/ChestXRay2017.zip' -d '/content/d/dd/'

df = pd.read_csv('./covid-chestxray-dataset/metadata.csv')
selected_df = df[df.finding=="Pneumonia/Viral/COVID-19"]
selected_df = selected_df[(selected_df.view == "AP") | (selected_df.view == "PA")]

df = pd.read_csv('./covid-chestxray-dataset/metadata.csv')
selected_df = df[df.finding=="Pneumonia/Viral/COVID-19"]
selected_df = selected_df[(selected_df.view == "AP") | (selected_df.view == "PA")]

images = selected_df.filename.values.tolist()

os.makedirs('./COVID19-DATASET/train/covid19')
os.makedirs('./COVID19-DATASET/train/normal')

COVID_PATH = './COVID19-DATASET/train/covid19'
NORMAL_PATH = './COVID19-DATASET/train/normal'

for image in images:
    shutil.copy(os.path.join('./covid-chestxray-dataset/images', image), os.path.join(COVID_PATH, image))

for image in os.listdir('/content/d/dd/chest_xray/train/NORMAL')[:300]:
    shutil.copy(os.path.join('/content/d/dd/chest_xray/train/NORMAL', image), os.path.join(NORMAL_PATH, image))

TRAIN_PATH = '/content/COVID19-DATASET/train'

print("TRAIN_PATH: {}".format(TRAIN_PATH))

!unzip -q '/content/drive/MyDrive/MEDICAL/COVID-19_Radiography_Database.zip' -d '/content/covid19-radiography-database/'

os.makedirs('./COVID19-DATASET/test/covid19')
os.makedirs('./COVID19-DATASET/test/normal')

COVID_TEST = '/content/covid19-radiography-database/COVID-19_Radiography_Dataset/COVID'
NORMAL_TEST = '/content/covid19-radiography-database/COVID-19_Radiography_Dataset/Normal'

for image in os.listdir(COVID_TEST)[:100]:
    shutil.copy(os.path.join(COVID_TEST, image), os.path.join('./COVID19-DATASET/test/covid19', image))
for image in os.listdir(NORMAL_TEST)[:100]:
    shutil.copy(os.path.join(NORMAL_TEST, image), os.path.join('./COVID19-DATASET/test/normal', image))

TEST_PATH = '/content/COVID19-DATASET/test'


Cloning into 'covid-chestxray-dataset'...
remote: Enumerating objects: 3641, done.
remote: Total 3641 (delta 0), reused 0 (delta 0), pack-reused 3641
Receiving objects: 100% (3641/3641), 632.96 MiB | 36.01 MiB/s, done.
Resolving deltas: 100% (1450/1450), done.
Checking out files: 100% (1174/1174), done.
TRAIN_PATH: /content/COVID19-DATASET/train


# Prepare for code

In [ ]:
import sys
!git clone https://github.com/Michaeltaoma/SecureCovid.git
%cd SecureCovid
!git checkout dev
sys.path.append("/content/SecureCovid")

Cloning into 'SecureCovid'...
remote: Enumerating objects: 110, done.
remote: Counting objects: 100% (110/110), done.
remote: Compressing objects: 100% (78/78), done.
remote: Total 110 (delta 53), reused 69 (delta 25), pack-reused 0
Receiving objects: 100% (110/110), 21.87 KiB | 10.94 MiB/s, done.
Resolving deltas: 100% (53/53), done.
/content/SecureCovid
Branch 'dev' set up to track remote branch 'dev' from 'origin'.
Switched to a new branch 'dev'


# import

In [ ]:
from model import pretrained
import util

import numpy as np

import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data.sampler import SubsetRandomSampler

import matplotlib.pyplot as plt
import time
import copy
from random import shuffle

import tqdm.notebook as tqdm

import sklearn
from sklearn.metrics import accuracy_score, cohen_kappa_score
from sklearn.metrics import classification_report
from PIL import Image
import cv2

In [ ]:
!python main.py --name "withOneFifty"

Training on GPU... Ready for HyperJump...
Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth
100% 30.8M/30.8M [00:00<00:00, 70.1MB/s]
Epoch 1/10
----------
train:   0%|          | 0/65 [00:00<?, ?it/s]
train Loss: 0.3917 | train Accuracy: 0.9027
val:   0%|          | 0/16 [00:00<?, ?it/s]
val Loss: 0.1649 | val Accuracy: 0.9609 | Kappa Score: 0.9196
Val loss Decreased from inf to 0.1649 
Saving Weights... 

Epoch 2/10
----------
train:   0%|          | 0/65 [00:00<?, ?it/s]
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:663: RuntimeWarning: invalid value encountered in true_divide
  k = np.sum(w_mat * confusion) / np.sum(w_mat * expected)
train Loss: 0.2144 | train Accuracy: 0.9377
val:   0%|          | 0/16 [00:00<?, ?it/s]
val Loss: 0.1594 | val Accuracy: 0.9609 | Kappa Score: 0.9080
Val loss Decreased from 0.1649 to 0.1594 
Saving Weights... 

Epoch 3/10
----------
trai

# Load the trained model

So that <code>shadow</code> is the model to be used

In [ ]:
import preprocess
import train
from model import pretrained

trainloader, valloader, dataset_size = preprocess.load_split_train_test(TRAIN_PATH, .2)
class_names = trainloader.dataset.classes
dataloaders = {"train": trainloader, "val": valloader}
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

In [ ]:
saved_path = "/content/drive/MyDrive/MEDICAL/trained/withOneFifty_1647134337.836162.pth"
shadow = pretrained.dense_shadow(device, class_names)
shadow.load_state_dict(torch.load(saved_path))
# shadow.eval()

<All keys matched successfully>

# Gather Attack train set

In [ ]:
from pathlib import Path
attack_train_set_path = Path("/content/drive/MyDrive/MEDICAL/attack_train")

In [ ]:
test_images = datasets.ImageFolder(TEST_PATH, transform=preprocess.test_transforms)
testloader = torch.utils.data.DataLoader(test_images, batch_size = 1)

In [ ]:
y_pred_list = []
y_true_list = []

out_x = []
out_y_pred = []
out_y_label = []
out = []

with torch.no_grad():
    for x_batch, y_batch in tqdm.tqdm(testloader, leave=False):
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        y_test_pred = shadow(x_batch)
        y_test_pred = torch.log_softmax(y_test_pred, dim=1)
        _, y_pred_tag = torch.max(y_test_pred, dim = 1)
        y_pred_list.append(y_pred_tag.cpu().numpy())
        y_true_list.append(y_batch.cpu().numpy())

        out_x.append(x_batch.cpu().numpy())
        out_y_pred.append(y_test_pred.cpu().numpy())
        out_y_label.append(y_batch.cpu().numpy())
        out.append(0)

  0%|          | 0/200 [00:00<?, ?it/s]

In [ ]:
def load_all_train(datadir, valid_size=0.0):
    train_data = datasets.ImageFolder(datadir,
                                      transform=preprocess.data_transforms['train'])
    test_data = datasets.ImageFolder(datadir,
                                     transform=preprocess.data_transforms['val'])
    num_train = len(train_data)
    indices = list(range(num_train))
    split = int(np.floor(valid_size * num_train))
    np.random.shuffle(indices)
    train_idx, test_idx = indices[split:], indices[:split]
    dataset_size = {"train": len(train_idx), "val": len(test_idx)}
    train_sampler = SubsetRandomSampler(train_idx)
    test_sampler = SubsetRandomSampler(test_idx)
    trainloader = torch.utils.data.DataLoader(train_data,
                                              sampler=train_sampler, batch_size=1)
    return trainloader

In [ ]:
all_train = load_all_train(TRAIN_PATH)

y_pred_list = []
y_true_list = []

with torch.no_grad():
    for x_batch, y_batch in tqdm.tqdm(all_train, leave=False):
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        y_test_pred = shadow(x_batch)
        y_test_pred = torch.log_softmax(y_test_pred, dim=1)
        _, y_pred_tag = torch.max(y_test_pred, dim = 1)
        y_pred_list.append(y_pred_tag.cpu().numpy())
        y_true_list.append(y_batch.cpu().numpy())

        out_x.append(x_batch.cpu().numpy())
        out_y_pred.append(y_test_pred.cpu().numpy())
        out_y_label.append(y_batch.cpu().numpy())
        out.append(1)

  0%|          | 0/642 [00:00<?, ?it/s]

In [ ]:
util.toPickle(out_y_label, attack_train_set_path.joinpath("{}.pkl".format("y_label")))
util.toPickle(out_y_pred, attack_train_set_path.joinpath("{}.pkl".format("y_pred")))
util.toPickle(out, attack_train_set_path.joinpath("{}.pkl".format("target")))

In [ ]:
if (out_y_label[0] != 0):
    print(True)

# Partition

In [ ]:
no_covid_y_pred = []
no_covid_target = []

covid_y_pred = []
covid_target = []


for label, pred, target in zip(out_y_label, out_y_pred, out):
    if label == 0:
        no_covid_y_pred.append(pred)
        no_covid_target.append(target)
    else:
        covid_y_pred.append(pred)
        covid_target.append(target)

In [82]:
partition_path = attack_train_set_path.joinpath("partition")

util.toPickle(no_covid_y_pred, partition_path.joinpath("{}.pkl".format("no_covid_y_pred")))
util.toPickle(no_covid_target, partition_path.joinpath("{}.pkl".format("no_covid_target")))

util.toPickle(covid_y_pred, partition_path.joinpath("{}.pkl".format("covid_y_pred")))
util.toPickle(covid_target, partition_path.joinpath("{}.pkl".format("covid_target")))